# Scikit-Learn Perceptron & Adaline Implemantations

In this section we will use the `linear_model` module from the `sklearn` library to predict if the individual will earn over 50K or not. We will use the hyperparameters we found in the previous scratch implementations in this notebook. 

In [19]:
# scikit-learn pereptron and adaline implementations
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import Perceptron as SkPerceptron, SGDRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer

---

## Load Processed Datasets

Read the preprocessed train/test feature matrices and labels, plus the validation features. These CSVs were produced earlier by the preprocessing pipeline (one-hot + numeric scaling), and will be the consistent input for all models.

In [20]:
# Reload features
X_train = pd.read_csv("../data/processed/X_train.csv")
X_test  = pd.read_csv("../data/processed/X_test.csv")
X_val = pd.read_csv("../data/processed/X_val.csv")

# Reload targets (squeezed into Series)
y_train = pd.read_csv("../data/processed/y_train.csv").squeeze("columns")
y_test  = pd.read_csv("../data/processed/y_test.csv").squeeze("columns")

# Align one-hot columns (safety)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_val  = X_val.reindex(columns=X_train.columns,  fill_value=0)

In [21]:
# To numpy
Xtr = X_train.to_numpy(dtype=np.float64, copy=False)
ytr_ppn = y_train.to_numpy(dtype=np.int64,  copy=False)   # Perceptron wants class labels
ytr_ada = y_train.to_numpy(dtype=np.float64, copy=False)  # Adaline learns on 0.0/1.0
Xte = X_test.to_numpy(dtype=np.float64,  copy=False)
yte = y_test.to_numpy(dtype=np.int64,    copy=False)
Xv  = X_val.to_numpy(dtype=np.float64,   copy=False)

----

## Hyperparameter Tunning

Now we will systematically find the optimal combination of hyperparameters.

### Perceptron (scikit-learn, GridSearchCV)

In [22]:
# GridSearchCV for best hyperparameters for SkPerceptron

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

ppn = SkPerceptron(random_state=42)
ppn_param_grid = {
    "penalty": [None, "l2", "l1", "elasticnet"],
    "alpha": [1e-5, 1e-4, 1e-3],
    "eta0": [0.01, 0.1, 1.0],      # learning rate for constant schedule
    "max_iter": [1000, 2000],
    "early_stopping": [True, False],
}

ppn_gs = GridSearchCV(
    estimator=ppn,
    param_grid=ppn_param_grid,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,
    refit=True,
    verbose=0,
)
ppn_gs.fit(Xtr, ytr_ppn)

print("Perceptron best params:", ppn_gs.best_params_)
print("Perceptron CV best accuracy:", ppn_gs.best_score_)
ppn_test_acc = accuracy_score(yte, ppn_gs.predict(Xte))
print("Perceptron TEST accuracy:", ppn_test_acc)

Perceptron best params: {'alpha': 0.0001, 'early_stopping': True, 'eta0': 0.1, 'max_iter': 1000, 'penalty': 'elasticnet'}
Perceptron CV best accuracy: 0.8191743347089357
Perceptron TEST accuracy: 0.7676263595649392


The grid search selected `alpha=0.0001`, `early_stopping=True`, `eta0=0.1`, `max_iter=1000`, and `penalty='elasticnet'` as the best parameters. With these settings, the model achieved a cross-validation accuracy of ~0.82, but the test accuracy dropped to ~0.77, indicating some overfitting.

### Adaline (scikit-learn, GridSearchCV)

In [23]:
# GridSearchCV for best hyperparameters for SGDRegressor (Adaline)

def adaline_acc_scorer(estimator, X, y):
    y_cont = estimator.predict(X)
    y_hat = (y_cont >= 0.5).astype(int)
    return accuracy_score(y, y_hat)

ada = SGDRegressor(
    loss="squared_error",
    learning_rate="constant",
    random_state=42,
)

ada_param_grid = {
    "penalty": [None, "l2"],
    "alpha": [1e-3, 1e-4, 1e-5],
    "eta0": [0.0001, 0.001, 0.01],
    "max_iter": [1000],
    "tol": [1e-3],
}

ada_gs = GridSearchCV(
    estimator=ada,
    param_grid=ada_param_grid,
    scoring=make_scorer(adaline_acc_scorer, greater_is_better=True),
    cv=cv,
    n_jobs=-1,
    refit=True,
    verbose=0,
)
ada_gs.fit(Xtr, ytr_ada)

print("Adaline best params:", ada_gs.best_params_)
print("Adaline CV best accuracy:", ada_gs.best_score_)
yte_ada_hat = (ada_gs.predict(Xte) >= 0.5).astype(int)
ada_test_acc = accuracy_score(yte, yte_ada_hat)
print("Adaline TEST accuracy:", ada_test_acc)

c:\Users\jonat\.conda\envs\DASC41103\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Adaline best params: {'alpha': 0.001, 'eta0': 0.0001, 'max_iter': 1000, 'penalty': None, 'tol': 0.001}
Adaline CV best accuracy: nan
Adaline TEST accuracy: 0.8344209852847089


The grid search produced instability for many hyperparameter combinations, leading to non-finite (`NaN`) cross-validation scores. This is a known issue with `SGDRegressor` when the learning rate or regularization values are not well balanced. Despite these warnings, the best estimator (`alpha=0.001`, `eta0=0.0001`, `max_iter=1000`, `penalty=None`) trained successfully and achieved a test accuracy of ~0.83.

---

## Predicting Outputs for `project_validation_inputs`

Generating the required prediction files (0/1, one per line, no header) for `project_validation_inputs.csv`.

In [24]:
# Save Perceptron predictions (0/1)
pd.Series(ppn_gs.predict(Xv)).to_csv(
    "../outputs/Group_18_Perceptron_PredictedOutputs.csv", 
    index=False, header=False
)

# Save Adaline predictions (threshold at 0.5 → 0/1)
pd.Series((ada_gs.predict(Xv) >= 0.5).astype(int)).to_csv(
    "../outputs/Group_18_Adaline_PredictedOutputs.csv", 
    index=False, header=False
)